# Linear Regression
The goal of this notebook is to implement linear regression from scratch as practice to refresh my knowledge of the process.
The goal of using Linear Regression is to find parameters $\beta$ such that $X \beta = y$ where $X$ is our matrix of features and $y$ is our target variable. 

The example dataset for this notebook is the boston housing dataset from sklearn


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()

In [3]:
boston_X = boston['data']
boston_X_with_intercept = np.concatenate([np.ones((boston_X.shape[0], 1)), boston_X], axis= 1)
boston_y = boston['target']

The first way I will implement linear regression is using ordinary least squares and minimizing the sum of squared residuals. This can be solved with the formula: 
$$ \hat{\beta} = (X^\top X)^{-1} X^\top y $$
Where $\hat{\beta}$ is our estimated parameters. This assumes that the columns of $X$ are linearly independent and that there exists a linear relationship between $X$ and $y$

In [4]:
#Solves for ordinary least squares
def basic_linear_regression(X, y):
    #the inverse of X transpose times X 
    XTX_inv =  np.linalg.inv(np.dot(X.T, X))
    #times X transpose times y
    beta_hat = np.dot(np.dot(XTX_inv, X.T), y)
    return beta_hat

This function will implement the cost function, the root mean square error
$$ L(\hat{y},y) = \sqrt{\frac{1}{N}\sum_{i = 1}^n (y_i - \hat{y_i})^2} $$
This will be the cost function that I will use to evaluate my implementations.

In [5]:
# Implement the cost function
def cost_function(y, y_hat):
    mean_squares = np.mean((np.array(y) - np.array(y_hat))**2)
    RMSE = np.sqrt(mean_squares)
    return RMSE

Now I'll use my implementation and see the RMSE, and compare it to scikit-learns implementation, using ten fold cross validation

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [7]:
my_im_train_error,my_im_test_error = [],[]
sklearn_train_error, sklearn_test_error = [],[]

kf = KFold(n_splits=10, random_state=42, shuffle = True)
for train_ind, test_ind in kf.split(boston_X_with_intercept):
    X_train, X_test = boston_X_with_intercept[train_ind], boston_X_with_intercept[test_ind]
    y_train, y_test = boston_y[train_ind], boston_y[test_ind]
    
    #estimates beta_hat for current fold
    beta_hat = basic_linear_regression(X_train, y_train)
    y_train_hat = X_train.dot(beta_hat)
    my_im_train_error.append(cost_function(y_train, y_train_hat))
                             
    y_test_hat = X_test.dot(beta_hat)
    my_im_test_error.append(cost_function(y_test, y_test_hat))
    
    #fit sklearns lr on current fold
    lr = LinearRegression()
    lr.fit(X_train,y_train)
                            
    y_train_hat = lr.predict(X_train)
    sklearn_train_error.append(cost_function(y_train, y_train_hat))
    
    y_test_hat = lr.predict(X_test)
    sklearn_test_error.append(cost_function(y_test, y_test_hat))
    
#average errors across folds
print("Average Errors on 10 folds")
print('My implementation')
print('train RMSE: ', np.mean(my_im_train_error))
print('test RMSE: ', np.mean(my_im_test_error))                            
print('sklearn')
print('train RMSE: ', np.mean(sklearn_train_error))
print('test RMSE: ', np.mean(sklearn_test_error))

Average Errors on 10 folds
My implementation
train RMSE:  4.67079221867
test RMSE:  4.79290191691
sklearn
train RMSE:  4.67079221867
test RMSE:  4.79290191691


In [8]:
boston_y.std()

9.1880115452782025

The results were exactly the same as sklearns implementation. This simple model has a RMSE of about 4.8 which is a little bit more than one half of a standard deviation on the target set. 

# Regularization: Ridge Regression

Linear regression can have a high variance, especially when fit on high dimensional data. To combat this we can restrict the model through regularization to avoid overfitting. The way I am going to implement this is through Ridge Regression. 

Ridge regression adds a penalty to the cost function for each coefficient in $\hat{\beta}$ to reduce the variance of the model.

Ridge Regression changes the cost function from 
$$ \sum_{i = 1}^m (y_i - X \hat{\beta})^2 $$

to 

$$ \sum_{i = 1}^m (y_i - X \hat{\beta})^2 + \lambda \sum_{i=1}^p \hat{\beta}_i^2 $$

where lambda is a hyperparameter that determines how much regularization to add. When $\lambda$ is 0 the cost function is identical to the sum of sqares, but as it increases the function will be more and more different

To optimize the choice of parameters $\hat{\beta}$ I will use gradient descent. Each parameter $\hat{\beta}_j$ will be updated by subtracting the gradient times a learning rate constant $\alpha$ after many iterations the parameter estimations will approach the minimum. The rules for updating the parameters are:
$$ \hat{\beta}_j = \hat{\beta}_j - \alpha(\sum_{i=1}^m (y_i - x_i \hat{\beta})x_i^{(j)} + 2\lambda\hat{\beta}_j)
$$ 

or for $j = 0$:

$$ \hat{\beta}_0 = \hat{\beta}_0 - \alpha(\frac{1}{m}\sum_{i=1}^m (y_i - x_i \hat{\beta}))
$$ 

where $\hat{y}_i$ is the prediction for training example $i$ and $\hat{y}_i = x_i\hat{\beta}$ and $x^{(j)}$ is the $j^{th}$ column of $X$


In [52]:
#completes one iteration of gradient descent
def find_gradient(X, y, beta, lmda):  
    m = X.shape[0]
    y_hat = np.dot(X, beta).flatten()
    grad  = np.zeros(beta.shape)
    for j in range(1, X.shape[1]):
        grad[j] =  np.sum( (y - y_hat) * -X[:,j]) + (lmda) * (beta[j])
    grad[0] = np.sum((y - y_hat))
    return grad

In [53]:
#completes one iteration of gradient descent
def iteration(X, y, beta, alpha ,lmda):
    gradient = find_gradient(X, y, beta, lmda)
    beta = beta  - (alpha * gradient)
    return beta

In [54]:

def ridge_regression(X, y, num_steps, alpha, lmda):
    beta = np.zeros((X.shape[1], 1))
    for i in range(num_steps):
        #if i% 10000 ==0:
            #print("iteration {} RMSE: {}".format(i,cost_function(y,np.dot(X,beta))))
        beta = iteration(X, y, beta, alpha, lmda)
    return beta  

In [55]:
ridge_regression(X_train,y_train,1000,.001,0.1)

array([[ -3.72447537e+02],
       [ -5.80569605e+00],
       [  3.18474862e+01],
       [ -4.18040636e+00],
       [  2.19364277e-01],
       [  1.55431757e-01],
       [  5.53097573e+00],
       [  4.02909790e+00],
       [  2.44406664e+00],
       [ -9.61398075e-01],
       [ -1.41825173e-01],
       [  4.43696800e+00],
       [  4.32800301e+01],
       [ -1.40699126e+01]])

In [56]:
from sklearn.linear_model import Ridge

my_im_train_error, my_im_test_error = [], []
sklearn_train_error, sklearn_test_error = [], []

#normalizing X t
boston_X_normalized = boston_X_with_intercept/boston_X_with_intercept.max()

lmda, alpha, num_iters = .1, .001, 1000

kf = KFold(n_splits=10, random_state=42, shuffle = True)
for train_ind, test_ind in kf.split(boston_X_with_intercept):
    #normalizing X other than the intercept
    #splitting into train test on this fold
    X_train, X_test = boston_X_normalized[train_ind], boston_X_normalized[test_ind]
    y_train, y_test = boston_y[train_ind], boston_y[test_ind]
    
    #estimates beta_hat for current fold
    beta_hat = ridge_regression(X_train, y_train, num_iters, alpha, lmda)
    y_train_hat = X_train.dot(beta_hat)
    my_im_train_error.append(cost_function(y_train, y_train_hat))
                             
    y_test_hat = X_test.dot(beta_hat)
    my_im_test_error.append(cost_function(y_test,y_test_hat))
    
    #fit sklearns lr on current fold
    lr = Ridge(lmda)
    lr.fit(X_train,y_train)
                            
    y_train_hat = lr.predict(X_train)
    sklearn_train_error.append(cost_function(y_train,y_train_hat))
    
    y_test_hat = lr.predict(X_test)
    sklearn_test_error.append(cost_function(y_test,y_test_hat))
    
#average errors across folds
print("Average Errors on 10 folds")
print('My implementation')
print('train RMSE: ',np.mean(my_im_train_error))
print('test RMSE: ',np.mean(my_im_test_error))                            
print('sklearn')
print('train RMSE: ',np.mean(sklearn_train_error))
print('test RMSE: ',np.mean(sklearn_test_error))

Average Errors on 10 folds
My implementation
train RMSE:  10.8505851619
test RMSE:  10.8130234167
sklearn
train RMSE:  7.06182805198
test RMSE:  7.10648385283


In [57]:
from sklearn.linear_model import Ridge
    

In [ ]:
lr = Ridge()